<a href="https://colab.research.google.com/github/petercloud23/Equipe_DataScience_SENAI_Out25/blob/main/UploadToDrive_ChooseFiles_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PRIMEIRPO PASSO: CRIE UMA NOVA CREDENCIAL OAUTH DO GOOGLE API, BAIXE O NOVO JSON E ADICIONE À PASTA PROJETO.

In [ ]:
import os
import io
import requests
import streamlit as st # Importa a biblioteca Streamlit para criar a interface web
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload

# --- Escopos ---
# Define os escopos de permissão necessários para acessar o Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# --- Autenticação ---
# Função para autenticar o usuário com a API do Google Drive
def autenticar_google():
    creds = None
    # Verifica se existe um arquivo 'token.json' com credenciais salvas
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Se não houver credenciais válidas, inicia o fluxo de autenticação
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # Se as credenciais expiraram, tenta atualizá-las
            creds.refresh(Request())
        else:
            # Se não há credenciais ou não podem ser atualizadas, inicia o fluxo OAuth 2.0
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Salva as credenciais em 'token.json' para uso futuro
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    # Constrói e retorna o serviço da API do Google Drive
    return build('drive', 'v3', credentials=creds)

# --- Listar pastas recursivamente ---
# Função para listar todas as pastas no Google Drive recursivamente
def listar_pastas_recursivo(service, parent_id=None, caminho_atual="Raiz"):
    pastas = {}
    page_token = None
    while True:
        # Lista os arquivos que são pastas e não estão na lixeira
        resposta = service.files().list(
            q=f"mimeType='application/vnd.google-apps.folder' and trashed=false" +
              (f" and '{parent_id}' in parents" if parent_id else ""),
            spaces='drive',
            fields="nextPageToken, files(id, name)",
            includeItemsFromAllDrives=True,
            supportsAllDrives=True,
            pageSize=200,
            pageToken=page_token
        ).execute()
        for file in resposta.get('files', []):
            # Constrói o caminho completo da pasta
            caminho_completo = f"{caminho_atual} / {file['name']}"
            pastas[caminho_completo] = file['id']
            # Recursivamente lista subpastas chamando a própria função
            subpastas = listar_pastas_recursivo(service, parent_id=file['id'], caminho_atual=caminho_completo)
            pastas.update(subpastas) # Adiciona as subpastas ao dicionário principal
        page_token = resposta.get('nextPageToken', None)
        if page_token is None:
            break # Sai do loop se não houver mais páginas
    return pastas # Retorna o dicionário com todas as pastas e seus IDs

# --- Upload ---
# Função para fazer upload de um arquivo para o Google Drive
def upload_para_drive(service, arquivo, pasta_id=None):
    # Define os metadados do arquivo (nome)
    file_metadata = {'name': arquivo.name}
    # Se uma pasta foi especificada, adiciona o ID da pasta aos metadados
    if pasta_id:
        file_metadata['parents'] = [pasta_id]
    # Cria um objeto MediaIoBaseUpload a partir do conteúdo do arquivo
    media = MediaIoBaseUpload(io.BytesIO(arquivo.getbuffer()), mimetype=arquivo.type)
    # Cria o arquivo no Google Drive usando a API
    file = service.files().create(body=file_metadata, media_body=media, fields='id, webViewLink').execute()
    return file # Retorna as informações do arquivo criado

# --- Desconectar conta ---
# Função para desconectar a conta do Google revogando o token e excluindo o arquivo local
def desconectar_conta():
    if os.path.exists("token.json"):
        try:
            # Carrega as credenciais do arquivo local
            creds = Credentials.from_authorized_user_file("token.json")
            # Envia uma requisição para revogar o token de acesso
            revoke = requests.post(
                'https://oauth2.googleapis.com/revoke',
                params={'token': creds.token},
                headers={'content-type': 'application/x-www-form-urlencoded'}
            )
            # Remove o arquivo token.json
            os.remove("token.json")
            if revoke.status_code == 200:
                st.success("✅ Conta desconectada com sucesso!")
            else:
                st.warning("⚠️ Falha ao revogar token, mas desconexão local concluída.")
        except Exception as e:
            st.error(f"Erro ao desconectar: {e}")
    else:
        st.info("Nenhuma conta autenticada no momento.")

# --- Interface ---
# Configura a página do Streamlit (título e ícone)
st.set_page_config(page_title="Upload para Google Drive", page_icon="☁️")
# Define o título principal da aplicação
st.title("☁️ Upload para Google Drive")

# Adiciona um markdown para explicar o uso da aplicação
st.markdown("""
Envie arquivos diretamente para seu **Google Drive**.
Escolha a pasta de destino usando o menu abaixo.
Agora você pode ver todas as subpastas com seus caminhos completos.
""")

# Autenticar e listar pastas
try:
    service = autenticar_google() # Autentica o usuário
    pastas_dict = {'Raiz do Drive': None}  # Cria um dicionário de pastas e sempre inclui a raiz
    pastas_dict.update(listar_pastas_recursivo(service)) # Adiciona as outras pastas listadas
except Exception as e:
    st.error(f"Erro na autenticação ou listagem de pastas: {e}")
    st.stop() # Para a execução em caso de erro

# Menu suspenso com caminhos completos das pastas
if pastas_dict:
    # Cria uma caixa de seleção (dropdown) com os nomes das pastas como opções
    pasta_selecionada_nome = st.selectbox(
        "📁 Selecione a pasta de destino:",
        options=list(pastas_dict.keys()) # Usa as chaves (nomes das pastas) do dicionário como opções
    )
    # Obtém o ID da pasta selecionada a partir do dicionário
    pasta_id = pastas_dict.get(pasta_selecionada_nome, None)
else:
    st.warning("Nenhuma pasta encontrada além da raiz.")
    pasta_selecionada_nome = "Raiz do Drive"
    pasta_id = None

# Upload de arquivo
# Cria um uploader de arquivo no Streamlit
arquivo = st.file_uploader("Escolha um arquivo para enviar", type=None)

# Botão para iniciar o upload
if st.button("📤 Enviar para Drive"):
    if arquivo: # Verifica se um arquivo foi selecionado
        if pasta_id is not None or pasta_selecionada_nome == "Raiz do Drive": # Verifica se uma pasta válida foi selecionada
            try:
                # Chama a função de upload
                file_info = upload_para_drive(service, arquivo, pasta_id)
                # Exibe mensagem de sucesso e o link para o arquivo no Drive
                st.success(f"✅ Arquivo enviado com sucesso!\n\n📁 ID: `{file_info.get('id')}`")
                st.markdown(f"[🔗 Abrir no Google Drive]({file_info.get('webViewLink')})")
            except Exception as e:
                st.error(f"Ocorreu um erro: {e}")
        else:
            st.warning("Selecione uma pasta válida para enviar o arquivo.")
    else:
        st.warning("Por favor, selecione um arquivo primeiro.")

# Linha divisória
st.divider()

# Botão de desconexão
if st.button("🔒 Desconectar conta Google"):
    desconectar_conta() # Chama a função para desconectar a conta

# Adiciona um rodapé com informações
st.markdown("---")
st.caption("Desenvolvido com ❤️ em Python + Streamlit + Google Drive API")

Agora abra o Prompt de Comandos e rode os seguintes códigos:

In [ ]:
#Acesse o diretório exato do seu Projeto, como no exemplo:
cd  C:\Users\Instrutor\PycharmProjects\PythonProject6 #localize seu prórpio caminho de diretórios

In [ ]:
#Crie o ambiente virtual do Python no CMD:
python -m venv venv

#Ativar o ambiente virtual:
venv\Scripts\activate

#Se estiver usando PowerShell:
venv\Scripts\Activate.ps1

#Quando o venv está ativo, o prompt mostra algo como (venv) no início.
#Para desativar (somente após terminar de usar o seu código):
deactivate

In [ ]:
#Instale o streamlit
python -m pip install streamlit

In [ ]:
#Instalar os pacotes necessários da API OAuth do Google

#Execute este comando no mesmo terminal onde você rodou o Streamlit:

python -m pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

'''
Esses quatro pacotes são os que o código precisa para:

autenticar (google-auth, google-auth-oauthlib),

criar a sessão segura (google-auth-httplib2),

e usar o Drive (google-api-python-client).
'''

Depois da instalação, digite:

In [ ]:
python -m pip list

'''
verifique se aparecem linhas como:
google-api-python-client
google-auth
google-auth-httplib2
google-auth-oauthlib
'''

Agora rode o streamlit via Prompt:

In [ ]:
streamlit run ARQUIVO FONTE.py

#O navegador abrirá automaticamente em http://localhost:8501